In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from rts.features.pose import load_all_poses, draw_pose
import umap
import cv2

from sklearn.cluster import DBSCAN, KMeans

poses_folder = "data/test_poses/"
sequences_folder = "/mnt/g/ioc/sequences/"

In [ ]:
poses = load_all_poses(poses_folder)

# Clustering

In [ ]:
reducer = umap.UMAP(n_neighbors=500, min_dist=0.1, metric='cosine')
pose_df = pd.DataFrame(poses)
embeddings = reducer.fit_transform(pose_df["angle_vec"].tolist())
pose_df["umap_x"] = embeddings[:,0]
pose_df["umap_y"] = embeddings[:,1]

In [ ]:
def cluster_poses(algo, pose_df, plot=True):
    algo.fit(pose_df[["umap_x", "umap_y"]])
    pose_df["label"] = algo.labels_
    if plot:
        plt.figure(figsize=(10,10))
        for label in pose_df["label"].unique():
            plt.scatter(pose_df[pose_df.label == label]["umap_x"], 
                        pose_df[pose_df.label == label]["umap_y"], 
                        s=1, label=label)
        plt.legend(markerscale = 5)
        plt.axis('off')
        plt.show()

    return pose_df

In [ ]:
kmeans = KMeans(n_clusters=25)
kmeans_poses = cluster_poses(kmeans, pose_df)

In [ ]:
dbscan = DBSCAN(eps=1, min_samples=10)
dbscan_poses = cluster_poses(dbscan, pose_df)

In [ ]:
# create a grid of subplots
nrows = 2
ncols = 6

for cluster in dbscan_poses["label"].unique()[:3]:
    print("Cluster", cluster)
    n_samples = dbscan_poses[dbscan_poses["label"] == cluster].shape[0]
    n_samples = min(n_samples, nrows * ncols)
    sample_poses = dbscan_poses[dbscan_poses["label"] == cluster].sample(n_samples).reset_index(drop=True)

    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 3, nrows * 3))
    axs = axs.flatten()
    # iterate over the poses and draw each pose in a subplot
    for i, pose in sample_poses.iterrows():
        draw_pose(pose, sequences_folder, ax = axs[i], cut = True)

    plt.show()
